# 线性回归

In [ ]:
import numpy as np

class LinearRegression:
    def __init__(self, learning_rate = 0.1, n_iters = 1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            y_pred = np.dot(X, self.weights) + self.bias
            
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db
    
    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

# 逻辑回归

In [ ]:
import numpy as np

class LogisticRegression:
    def __init__(self, learning_rate = 0.1, n_iters = 1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(- z))
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.random(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            linear = np.dot(X, self.weights) + self.bias
            y_pred = self.sigmoid(linear)

            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict_prob(self, X):
        linear = np.dot(X, self.weights) + self.bias
        return self.sigmoid(linear)
    
    def predict(self, X, threshold = 0.5):
        return (self.predict_prob(X) >= threshold).astype(int)

# Softmax 回归

In [ ]:
import numpy as np

class SoftMaxRegression:
    def __init__(self, n_classes, learning_rate = 0.01, n_iters = 1000):
        self.n_classes = n_classes
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def softmax(self, Z):
        exp_Z = np.exp(Z - max(Z, axis = 1, keepdim = True))
        return exp_Z / exp_Z.sum(axis = 1, keepdim = True)
    
    def one_hot(self, y):
        m = y.shape[0]
        y_one_hot = np.zeros((m, self.n_classes))
        y_one_hot[np.range(m), y] = 1
        return y_one_hot
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        y_one_hot = self.one_hot(y)
        self.weights = np.random((self.n_classes, n_features))
        self.bias = np.random(self.n_classes)

        for _ in range(self.n_iters):
            Z = np.dot(X, self.weights) + self.bias
            P = self.softmax(Z)

            dZ = P - y_one_hot
            dw = (1 / n_samples) * np.dot(dZ.T, X)
            db = (1 / n_samples) * np.dot(dZ, axis = 0)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db
        
    def predict(self, X):
        Z = np.dot(X, self.weights) + self.bias
        P = self.softmax(Z)
        return np.max(P, axis = 1)



# SGD 优化器

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split

class LinearRegressionSGD:
    def __init__(self, learning_rate = 0.001, n_iters = 1000, batch_size = 64, validate_every = 4):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.batch_size = batch_size
        self.validate_every = validate_every
        self.weights = None
        self.bias = None
        self.train_loss_history = []
        self.val_loss_history = []

    def add_bias_term(self, X):
        return np.hstack([X, np.ones((X.shape[0], 1))])
    
    def get_batches(self, X, y):
        n_samples = X.shape[0]
        for i in range(0, n_samples, self.batch_size):
            end = min(i + self.batch_size, n_samples)
            yield X[i : end], y[i : end]
    
    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

    def compute_loss(self, X, y):
        y_pred = self.predict(X)
        return 0.5 * np.mean((y_pred - y) ** 2)
    
    def compute_gradient(self, X, y):
        n_samples = X.shape[0]
        y_pred = self.predict(X)
        error = y_pred - y
        dw = np.dot(X.T, error) / n_samples
        db = np.mean(error)
        return dw, db

    def shuffled_data(self, X, y):
        n_samples = X.shape[0]
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        return X[indices], y[indices]

    def fit(self, X_train, y_train, X_val = None, y_val = None):
        X_train_bias = self.add_bias_term(X_train)
        n_features = X_train_bias.shape[1]
        n_samples = X_train.shape[0]
        self.weights = np.random.randn(n_features - 1)
        self.bias = 0

        for epoch in range(self.n_iters):
            epoch_loss = 0
            X_shuffled, y_shuffled = self.shuffled_data(X_train_bias, y_train)

            for X_batch, y_batch in self.get_batches(X_shuffled, y_shuffled):
                y_pred = np.dot(X_batch[:, :-1], self.weights) + self.bias

                batch_loss = 0.5 * np.mean((y_pred - y_batch) ** 2)
                epoch_loss += batch_loss * X_batch.shape[0] / n_samples
                dw, db = self.compute_gradient(X_batch[:, :-1], y_batch)
                self.weights -= self.lr * dw
                self.bias -= self.lr * db

            self.train_loss_history.append(epoch_loss)

            if X_val is not None and y_val is not None and (epoch % self.validate_every) == 0:
                X_val_bias = self.add_bias_term(X_val)
                val_loss = self.compute_loss(X_val_bias[:, :-1], y_val)
                self.val_loss_history.append(val_loss)
    
def generate_sample_data(n_samples = 2000, n_features = 5, noise_level = 0.8, random_seed = 9):
    np.random.seed(random_seed)
    X = np.random.randn(n_samples, n_features)
    true_weights = np.random.randn(n_features)
    true_bias = np.random.randn()

    y = np.dot(X, true_weights) + true_bias + noise_level * np.random.randn(n_samples)
    return X, y, true_weights, true_bias

if __name__ == "__main__":
    X, y, true_weights, true_bias = generate_sample_data()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle= True)

    model = LinearRegressionSGD()

    model.fit(X_train, y_train, X_test, y_test)

    print(f"Final training loss: {model.train_loss_history[-1]:.4f}")
    print(f"Final validation loss: {model.val_loss_history[-1]:.4f}")   

Final training loss: 0.3099
Final validation loss: 0.3071


# K-Means

In [ ]:
import numpy as np

def kmeans(data, k, threshold = 1, max_iterations = 100):
    centers = data[np.random.choice(data.shape[0], k, replace = False)]

    for _ in range(max_iterations):
        distances = np.linalg.norm(data[:, None] - centers, axis= 2)
        labels = np.argmin(distances, axis= 1)
        new_centers = np.array([data[labels == i].mean(axis = 0) for i in range(k)])

        if np.all(centers == new_centers):
            break
        
        centers = new_centers
    return labels, centers

data = np.random.rand(100, 2)
k = 3
labels, centers = kmeans(data, k)